##  Mastodon toot collection

In [18]:
from mastodon import Mastodon
from IPython.display import HTML
from datetime import datetime, timedelta, timezone
import pandas as pd
import re

### Mastadon Data Collection

In [19]:
Mastodon.create_app(
    'your_app_name',
    api_base_url = 'https://mastodon.social', #this is determined by your chosen server, if you picked the general server, don't change this line
    to_file = 'client_crediential_file_name.secret'
)

('RHR9EVT6E8bGqU_H3aV2csOHmZRpTt0SpeGSHvv9HFM',
 's4UWyKiFdm8XR9-C8e1Qmv8weq2y9MCc6Oo0-uulhPg')

In [20]:
API = Mastodon(client_id = 'client_crediential_file_name.secret',)
url = API.auth_request_url(scopes=["read", "write", "follow", "push"])
print("Please visit this URL to authorize the app:", url)

Please visit this URL to authorize the app: https://mastodon.social/oauth/authorize?client_id=RHR9EVT6E8bGqU_H3aV2csOHmZRpTt0SpeGSHvv9HFM&response_type=code&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=read+write+follow+push&force_login=False&state=None&lang=None


In [21]:
code = input("Enter the code from the browser: ")

In [22]:
API.log_in(
    code=code,
    to_file = 'user_crediential_file_name.secret'
)
print("Successfully logged in. User access token saved in 'apple_user.secret'.")

Successfully logged in. User access token saved in 'apple_user.secret'.


In [23]:
q0 = "ChatGPT"
q1 = "chatgpt"
q2 = "artifical intelligence"
q3 = "Artifical Intelligence"
mast_toots = []
ai_toots = []
off_set = 0

In [35]:
for i in [q0, q1]:
    while True:
        # Fetch toots with the search API using offset
        search_results = API.search_v2(q=i, result_type="statuses", offset=off_set)
        statuses = search_results["statuses"]

        if not statuses:
            print("Found all toots")
            break  # Exit loop if no more toots are returned

        # Add new toots to the collection
        mast_toots.extend(statuses)

        earliest_toot = min(statuses, key=lambda x: x["created_at"])

        if  timedelta(days=7) <= datetime.now(tz=timezone.utc) - earliest_toot["created_at"] and len(mast_toots) > 1000:
            print("Reached 1 week")
            break

        # Update offset to fetch the next batch of results
        off_set += len(statuses)

    # Print results
    print(f"Total toots collected: {len(mast_toots)}")
    if mast_toots:
        print(f"Earliest toot: {mast_toots[-1]['created_at']}")
        print(f"Latest toot: {mast_toots[0]['created_at']}")

Reached 1 week
Total toots collected: 4220
Earliest toot: 2024-12-08 18:29:37.794000+00:00
Latest toot: 2024-12-15 10:25:34+00:00
Reached 1 week
Total toots collected: 4237
Earliest toot: 2024-12-08 18:29:37.794000+00:00
Latest toot: 2024-12-15 10:25:34+00:00


In [36]:
off_set = 0
for i in [q2, q3]:
    while True:
        # Fetch toots with the search API using offset
        search_results = API.search_v2(q=i, result_type="statuses", offset=off_set)
        statuses = search_results["statuses"]

        if not statuses:
            print("found all toots")
            break  # Exit loop if no more toots are returned

        # Add new toots to the collection
        ai_toots.extend(statuses)

        earliest_toot = min(statuses, key=lambda x: x["created_at"])

        # Update offset to fetch the next batch of results
        off_set += len(statuses)

    # Print results
    print(f"Total toots collected: {len(ai_toots)}")
    if ai_toots:
        print(f"Earliest toot: {ai_toots[-1]['created_at']}")
        print(f"Latest toot: {ai_toots[0]['created_at']}")

found all toots
Total toots collected: 555
Earliest toot: 2018-08-21 06:30:29+00:00
Latest toot: 2024-12-12 11:12:01.740000+00:00
found all toots
Total toots collected: 555
Earliest toot: 2018-08-21 06:30:29+00:00
Latest toot: 2024-12-12 11:12:01.740000+00:00


In [37]:
needed_attributes = [
    'id', 'created_at', 'content', 'language', 'replies_count', 
    'reblogs_count', 'favourites_count', 'visibility', 'tags'
]

cleaned_mast_toots = []
cleaned_ai_toots = []

for toot in mast_toots:
    cleaned_toot = {key: toot[key] for key in needed_attributes if key in toot}
    cleaned_mast_toots.append(cleaned_toot)

for toot in ai_toots:
    cleaned_toot = {key: toot[key] for key in needed_attributes if key in toot}
    cleaned_ai_toots.append(cleaned_toot)
    

In [38]:
def strip_html_tags(html_text):
    plain_text = re.sub(r'<.*?>', '', html_text)
    return plain_text

for toot in cleaned_mast_toots:
    if 'content' in toot and toot['content']:  # Check if 'content' exists and is not None
        toot['content'] = strip_html_tags(toot['content'])

for toot in cleaned_ai_toots:
    if 'content' in toot and toot['content']:  # Check if 'content' exists and is not None
        toot['content'] = strip_html_tags(toot['content'])

In [39]:
mast_df = pd.DataFrame(cleaned_mast_toots)
ai_df = pd.DataFrame(cleaned_ai_toots)
ai_df

,id,created_at,content,language,replies_count,reblogs_count,favourites_count,visibility,tags
0,113639550502224256,2024-12-12 11:12:01.740000+00:00,Ainsi s'ouvrirent les portes de l'enfer.\nwww....,fr,0,0,0,public,[]
1,113639537305656138,2024-12-11 23:50:20+00:00,What if AI can actually help with your burnout...,en,0,0,0,public,[]
2,113637263387670537,2024-12-12 01:30:31.801000+00:00,@fbi @washingtonpost haha dude Biden is really...,en,0,0,0,public,[]
3,113629601136983194,2024-12-10 16:56:54+00:00,Oracle Just Shared Some Amazing Insight With A...,en,0,0,0,public,[]
4,113611582071097671,2024-12-07 12:35:00+00:00,Prediction: This Will Be 2025's Top-Performing...,en,0,0,0,public,[]
...,...,...,...,...,...,...,...,...,...
550,108187053885501189,2022-04-24 12:30:33+00:00,"@loveisgrief ja, klar gibts immer viel zu beri...",en,1,0,0,public,[]
551,107374987354924180,2021-12-02 02:31:22+00:00,Homeland security is requesting comments from ...,en,0,3,1,public,"[{'name': 'privacy', 'url': 'https://mastodon...."
552,103172496385073778,2019-11-20 22:02:37+00:00,"Perhaps they, given the increase of intelligen...",en,1,0,0,public,[]
553,101890578358125434,2019-04-08 12:33:56+00:00,"""UK businesses using artificial intelligence t...",en,0,1,0,public,"[{'name': 'ai', 'url': 'https://mastodon.socia..."


In [40]:
mast_df.to_csv('mast_toots.csv', index=False)
ai_df.to_csv('ai_toots.csv', index=False)